In [1]:
from utils import read_column_from_excel, MultiLinearRegression, get_onehot_labels

temperature1 = read_column_from_excel('附件一（训练集）.xlsx', column_name="温度，oC", sheet_name="材料1")
wave_shape1 = get_onehot_labels('附件一（训练集）.xlsx', column_name="励磁波形", name="材料1")


temperature2 = read_column_from_excel('附件一（训练集）.xlsx', column_name="温度，oC", sheet_name="材料2")
wave_shape2 = get_onehot_labels('附件一（训练集）.xlsx', column_name="励磁波形", name="材料2")

temperature3 = read_column_from_excel('附件一（训练集）.xlsx', column_name="温度，oC", sheet_name="材料3")
wave_shape3 = get_onehot_labels('附件一（训练集）.xlsx', column_name="励磁波形", name="材料3")

temperature4 = read_column_from_excel('附件一（训练集）.xlsx', column_name="温度，oC", sheet_name="材料4")
wave_shape4 = get_onehot_labels('附件一（训练集）.xlsx', column_name="励磁波形", name="材料4")

wave_shape1

,三角波,梯形波,正弦波
0,False,False,True
1,False,False,True
2,False,False,True
3,False,False,True
4,False,False,True
...,...,...,...
3395,False,True,False
3396,False,True,False
3397,False,True,False
3398,False,True,False


In [2]:
wave_shape4.shape

(2800, 3)

In [3]:
import numpy as np

# 创建四个 (3400, 4) 的 one-hot 矩阵
one_hot_matrix1 = np.zeros((3400, 4))
one_hot_matrix1[:, 0] = 1  

one_hot_matrix2 = np.zeros((3000, 4))
one_hot_matrix2[:, 1] = 1  

one_hot_matrix3 = np.zeros((3200, 4))
one_hot_matrix3[:, 2] = 1  

one_hot_matrix4 = np.zeros((2800, 4))
one_hot_matrix4[:, 3] = 1  

print(one_hot_matrix1)
# 拼接 
x1 = np.column_stack((temperature1, wave_shape1, one_hot_matrix1))
x2 = np.column_stack((temperature2, wave_shape2, one_hot_matrix2))
x3 = np.column_stack((temperature3, wave_shape3, one_hot_matrix3))
x4 = np.column_stack((temperature4, wave_shape4, one_hot_matrix4))

x4.shape

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


(2800, 8)

**输入变量拼接**

In [4]:
combined = np.vstack((x1, x2, x3, x4))
combined.shape

(12400, 8)

**拼接标签数据**

In [5]:
y1 = read_column_from_excel('附件一（训练集）.xlsx', column_name='磁芯损耗，w/m3', sheet_name='材料1')
y2 = read_column_from_excel('附件一（训练集）.xlsx', column_name='磁芯损耗，w/m3', sheet_name='材料2')
y3 = read_column_from_excel('附件一（训练集）.xlsx', column_name='磁芯损耗，w/m3', sheet_name='材料3')
y4 = read_column_from_excel('附件一（训练集）.xlsx', column_name='磁芯损耗，w/m3', sheet_name='材料4')
y4

0         2525.795337
1         4427.524958
2         5780.576037
3        10983.412600
4        14526.685680
            ...      
2795     20604.900210
2796     26504.284280
2797     46232.491530
2798     61458.085900
2799    107581.189300
Name: 磁芯损耗，w/m3, Length: 2800, dtype: float64

In [6]:
y4.shape

(2800,)

In [7]:
y_combined = np.concatenate((y1, y2, y3, y4))
y_combined.shape

(12400,)

**训练**

In [8]:

from sklearn.model_selection import train_test_split
# 多元逻辑回归
mlr = MultiLinearRegression()
X = combined
Y = y_combined

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
X

array([[25.,  0.,  0., ...,  0.,  0.,  0.],
       [25.,  0.,  0., ...,  0.,  0.,  0.],
       [25.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [90.,  0.,  1., ...,  0.,  0.,  1.],
       [90.,  0.,  1., ...,  0.,  0.,  1.],
       [90.,  0.,  1., ...,  0.,  0.,  1.]])

In [10]:
# 拟合
mlr.fit(X_train, Y_train)
# 评估
score = mlr.score(X_test, Y_test)
print(f"Model R² score: {score}")

Model R² score: 0.06229806687813377


In [11]:
from utils import RandomForestRegressorModel

# 创建模型实例
rf_model = RandomForestRegressorModel(n_estimators=100, random_state=42)

# 训练模型
rf_model.train(X_train, Y_train)

    # 进行预测
predictions = rf_model.predict(X_test)

# 评估模型
r2, mse = rf_model.evaluate(Y_test, predictions)

print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")


R² Score: 0.05818992851726035
Mean Squared Error: 126709761013.04173


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class NeuralNetworkRegressor(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetworkRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_model(model, criterion, optimizer, X_train, y_train, epochs=100):
    model.train()
    for epoch in range(epochs):
        inputs = torch.tensor(X_train, dtype=torch.float32)
        targets = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)        #修改
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X_test, dtype=torch.float32)
        outputs = model(inputs)
        return outputs.numpy()


# data = pd.read_excel('附件一.xlsx')
# X = data[['温度', '励磁波形1', '励磁波形2', '磁芯材料1', '磁芯材料2']]
# y = data['磁芯损耗']

# 划分训练集和测试集

    # 数据标准化
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# 创建模型实例
input_size = X_train.shape[1]
model = NeuralNetworkRegressor(input_size)
print("input size",input_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1)

# 训练模型
train_model(model, criterion, optimizer, X_train, Y_train, epochs=1000)

# 进行预测
predictions = evaluate_model(model, X_test, Y_test)

# 评估模型
from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(Y_test, predictions)
mse = mean_squared_error(Y_test, predictions)

print(f"Neural Network R² Score: {r2}")
print(f"Neural Network Mean Squared Error: {mse}")

input size 5
Epoch [10/1000], Loss: 143037579264.0000
Epoch [20/1000], Loss: 134006013952.0000
Epoch [30/1000], Loss: 132300931072.0000
Epoch [40/1000], Loss: 132101906432.0000
Epoch [50/1000], Loss: 132015161344.0000
Epoch [60/1000], Loss: 131900809216.0000
Epoch [70/1000], Loss: 131774611456.0000
Epoch [80/1000], Loss: 131722805248.0000
Epoch [90/1000], Loss: 131671924736.0000
Epoch [100/1000], Loss: 131630489600.0000
Epoch [110/1000], Loss: 131587383296.0000
Epoch [120/1000], Loss: 131549937664.0000
Epoch [130/1000], Loss: 131513483264.0000
Epoch [140/1000], Loss: 131477397504.0000
Epoch [150/1000], Loss: 131444998144.0000
Epoch [160/1000], Loss: 131416965120.0000
Epoch [170/1000], Loss: 131392323584.0000
Epoch [180/1000], Loss: 131368656896.0000
Epoch [190/1000], Loss: 131350962176.0000
Epoch [200/1000], Loss: 131334021120.0000
Epoch [210/1000], Loss: 131319291904.0000
Epoch [220/1000], Loss: 131305775104.0000
Epoch [230/1000], Loss: 131292839936.0000
Epoch [240/1000], Loss: 131279

In [18]:
# PCA
# import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 假设 X 是从 Excel 或其他来源读取的数据
# X = pd.read_excel('your_data.xlsx')

# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 应用 PCA
pca = PCA(n_components=5)  # 保留 2 个主成分
X_pca = pca.fit_transform(X_scaled)

# 查看解释的方差
explained_variance = pca.explained_variance_ratio_
print("Explained variance ratio:", explained_variance)

# X_pca 现在可以用于模型训练


Explained variance ratio: [0.21174993 0.18663396 0.17011099 0.16358272 0.14436184]


In [19]:
X_pca.shape

(12400, 5)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

# 拟合
mlr.fit(X_train, Y_train)
# 评估
score = mlr.score(X_test, Y_test)
print(f"Model R² score: {score}")

Model R² score: 0.05980967751461175
